## Data loading

### Subtask:
Load the two CSV files into pandas DataFrames.


In [ ]:
# from google.colab import files

# uploaded = files.upload()


**Reasoning**:
Load the two CSV files into pandas DataFrames.



In [ ]:
import pandas as pd

movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

## Data preparation

### Subtask:
Merge the two dataframes, `movies` and `credits`, based on the 'title' column.


**Reasoning**:
Merge the movies and credits dataframes on the 'title' column.



In [ ]:
movies = movies.merge(credits, on='title')

## Data wrangling

### Subtask:
Select relevant columns and extract information from specific columns using the provided helper function.


**Reasoning**:
Select the relevant columns and apply the extract_names function to the specified columns, modifying it for the 'crew' column to extract directors' names.



In [ ]:
# Keep only required columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Function to convert stringified list to list of names
def extract_names(obj, key='name', limit=None):
    try:
        items = ast.literal_eval(obj)
        if key == 'name':
            names = [item[key] for item in items]
            if limit:
                names = names[:limit]
            return names
        elif key == 'job' and isinstance(items, list):
            names = [item['name'] for item in items if item.get('job') == 'Director']
            return names
        return []
    except:
        return []

# Extract genres, keywords, cast (top 3), director
movies['genres'] = movies['genres'].apply(lambda x: extract_names(x))
movies['keywords'] = movies['keywords'].apply(lambda x: extract_names(x))
movies['cast'] = movies['cast'].apply(lambda x: extract_names(x, limit=3))
movies['crew'] = movies['crew'].apply(lambda x: extract_names(x, key='job'))

In [ ]:
import ast

# Keep only required columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Function to convert stringified list to list of names
def extract_names(obj, key='name', limit=None):
    try:
        items = ast.literal_eval(obj)
        if key == 'name':
            names = [item[key] for item in items]
            if limit:
                names = names[:limit]
            return names
        elif key == 'job' and isinstance(items, list):
            names = [item['name'] for item in items if item.get('job') == 'Director']
            return names
        return []
    except (ValueError, TypeError, IndexError):
        return []

# Extract genres, keywords, cast (top 3), director
movies.loc[:, 'genres'] = movies['genres'].apply(lambda x: extract_names(x))
movies.loc[:, 'keywords'] = movies['keywords'].apply(lambda x: extract_names(x))
movies.loc[:, 'cast'] = movies['cast'].apply(lambda x: extract_names(x, limit=3))
movies.loc[:, 'crew'] = movies['crew'].apply(lambda x: extract_names(x, key='job'))

**Reasoning**:
The error message indicates that the `ast` module is not imported.  I need to import it before using `ast.literal_eval`.



In [ ]:
import ast

# Load the original movie dataset to retrieve the required columns
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = movies.merge(credits, on='title')


# Now you can select the required columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# ... (rest of your code) ...
# Function to convert stringified list to list of names
def extract_names(obj, key='name', limit=None):
    try:
        items = ast.literal_eval(obj)
        if key == 'name':
            names = [item[key] for item in items]
            if limit:
                names = names[:limit]
            return names
        elif key == 'job' and isinstance(items, list):
            names = [item['name'] for item in items if item.get('job') == 'Director']
            return names
        return []
    except (ValueError, TypeError, IndexError):
        return []

# Extract genres, keywords, cast (top 3), director
movies.loc[:, 'genres'] = movies['genres'].apply(lambda x: extract_names(x))
movies.loc[:, 'keywords'] = movies['keywords'].apply(lambda x: extract_names(x))
movies.loc[:, 'cast'] = movies['cast'].apply(lambda x: extract_names(x, limit=3))
movies.loc[:, 'crew'] = movies['crew'].apply(lambda x: extract_names(x, key='job'))

## Feature engineering

### Subtask:
Combine the relevant text columns ('overview', 'genres', 'keywords', 'cast', 'crew') into a new 'tags' column.


**Reasoning**:
Combine the relevant text columns ('overview', 'genres', 'keywords', 'cast', 'crew') into a new 'tags' column, handling missing values in 'overview'.



In [ ]:
# Fill missing values in 'overview' with empty strings
movies['overview'] = movies['overview'].fillna('')

# Combine text fields into a single 'tags' column
movies['tags'] = movies['overview'] + ' ' + \
                 movies['genres'].apply(lambda x: ' '.join(x)) + ' ' + \
                 movies['keywords'].apply(lambda x: ' '.join(x)) + ' ' + \
                 movies['cast'].apply(lambda x: ' '.join(x)) + ' ' + \
                 movies['crew'].apply(lambda x: ' '.join(x))

## Data preparation

### Subtask:
Drop unnecessary columns from the DataFrame, leaving only 'movie_id', 'title', and 'tags'.


**Reasoning**:
Select only the required columns 'movie_id', 'title', and 'tags' from the movies DataFrame.



In [ ]:
movies = movies[['movie_id', 'title', 'tags']]

## Summary:

### 1. Q&A

No questions were posed in the provided task description.


### 2. Data Analysis Key Findings

*   The datasets `tmdb_5000_movies.csv` and `tmdb_5000_credits.csv` were successfully merged based on the 'title' column.
*   A new 'tags' column was created by combining 'overview', 'genres', 'keywords', 'cast', and 'crew' information, creating a text-based representation of each movie for content-based filtering.
*   The 'cast' column was limited to the top 3 actors for each movie.
*   Only the director's name was extracted from the 'crew' column.
*   Missing values in the 'overview' column were filled with empty strings.


### 3. Insights or Next Steps

*   The 'tags' column is now ready for text vectorization techniques (TF-IDF, word embeddings) to build a content-based movie recommendation system.
*   Explore other features like budget, revenue, popularity, and vote counts to incorporate into the recommendation model for a more comprehensive approach.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

def build_movie_recommendation_model():
    """Builds and saves the movie recommendation model."""

    # Download NLTK resources if not already downloaded
    nltk.download('stopwords')

    # Load datasets
    movies = pd.read_csv("tmdb_5000_movies.csv")
    credits = pd.read_csv("tmdb_5000_credits.csv")
    movies = movies.merge(credits, on='title')

    # Keep only required columns
    movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']].copy() # Use .copy() to avoid SettingWithCopyWarning

    # Drop rows with missing titles
    movies.dropna(subset=['title'], inplace=True)

    # Function to convert stringified list to list of names
    def extract_names(obj, key='name', limit=None):
        try:
            items = ast.literal_eval(obj)
            if key == 'name':
                names = [item[key] for item in items]
                if limit:
                    names = names[:limit]
                return names
            elif key == 'job' and isinstance(items, list):
                names = [item['name'] for item in items if item.get('job') == 'Director']
                return names
            return []
        except (ValueError, TypeError, IndexError):
            return []

    # Extract genres, keywords, cast (top 3), director
    movies.loc[:, 'genres'] = movies['genres'].apply(lambda x: extract_names(x))
    movies.loc[:, 'keywords'] = movies['keywords'].apply(lambda x: extract_names(x))
    movies.loc[:, 'cast'] = movies['cast'].apply(lambda x: extract_names(x, limit=3))
    movies.loc[:, 'crew'] = movies['crew'].apply(lambda x: extract_names(x, key='job'))

    # Fill missing values in 'overview' with empty strings
    movies['overview'] = movies['overview'].fillna('')

    # Combine text fields into a single 'tags' column
    movies['tags'] = movies['overview'] + ' ' + \
                    movies['genres'].apply(lambda x: ' '.join(x)) + ' ' + \
                    movies['keywords'].apply(lambda x: ' '.join(x)) + ' ' + \
                    movies['cast'].apply(lambda x: ' '.join(x)) + ' ' + \
                    movies['crew'].apply(lambda x: ' '.join(x))

    # Data Cleaning and Preprocessing
    ps = PorterStemmer()

    def clean_tags(text):
        text = re.sub('[^a-zA-Z0-9 ]', '', text)
        text = text.lower()
        text = ' '.join([ps.stem(word) for word in text.split() if word not in stopwords.words('english')])
        return text

    movies['tags'] = movies['tags'].apply(clean_tags)

    # Text Vectorization and Similarity Calculation
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movies['tags'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Create the indices mapping
    indices = pd.Series(movies.index, index=movies['title'].apply(lambda x: re.sub(r'[- ]', '', x).lower())).drop_duplicates()

    # --- Modified Saving for compatibility with api.py ---
    model_data = {
        'cosine_sim': cosine_sim,
        'movies': movies[['title', 'overview']].to_dict(orient='records'), # Save relevant movie info
        'indices': indices.to_dict()
    }

    with open('movie_recommendation_model.pkl', 'wb') as f:
        pickle.dump(model_data, f)

    print("Model data saved successfully to 'movie_recommendation_model.pkl' in the format expected by the API.")

def get_recommendations(title, model_path='movie_recommendation_model.pkl'):
    """Loads the model and generates movie recommendations, handling missing titles and ignoring characters."""

    with open(model_path, 'rb') as f:
        model_data = pickle.load(f)

    cosine_sim = model_data['cosine_sim']
    movies_list = model_data['movies']
    indices_dict = model_data['indices']
    movies_df = pd.DataFrame(movies_list)
    indices = pd.Series(indices_dict)

    processed_title = re.sub(r'[- ]', '', title).lower()

    if processed_title not in indices:
        print(f"Movie title '{title}' not found. Please try a different spelling.")
        return None

    idx = indices[processed_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies_df['title'].iloc[movie_indices].tolist()

# Build the model when the script is run
if __name__ == "__main__":
    build_movie_recommendation_model()

    # --- Testing with loop for invalid titles ---
    while True:
        title = input("Enter a movie title: ")
        recommendations = get_recommendations(title)

        if recommendations is not None:  # Recommendations were generated
            print("Recommendations:")
            for rec in recommendations:
                print(f"- {rec}")
            break  # Exit the loop

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Model data saved successfully to 'movie_recommendation_model.pkl' in the format expected by the API.
Enter a movie title: tangled
Recommendations:
- Into the Woods
- Enchanted
- The NeverEnding Story
- The Green Mile
- Ella Enchanted
- Happily N'Ever After
- A Simple Wish
- The Princess and the Frog
- The Horse Whisperer
- Pocahontas


In [ ]:
# Assuming you have your 'get_recommendations' function defined
recommendations = get_recommendations('Tangled')
print(recommendations)  # Check if the recommendations are similar in genre, theme, etc.

['Into the Woods', 'Enchanted', 'The NeverEnding Story', 'The Green Mile', 'Ella Enchanted', "Happily N'Ever After", 'A Simple Wish', 'The Princess and the Frog', 'The Horse Whisperer', 'Pocahontas']
